## Deliverable 3. Create a Travel Itinerary Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps
gmaps.configure(api_key=g_key)

In [2]:
# 1. Read the WeatherPy_vacation.csv into a DataFrame.
vacation_df = pd.read_csv("../Vacation_Search/WeatherPy_vacation.csv")
vacation_df.head(10)

,City_ID,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
0,1,Hilo,US,76.73,overcast clouds,19.7297,-155.0900,Hilo Hawaiian Hotel
1,6,Jieshi,CN,82.00,light rain,22.8134,115.8257,Yuehai Hotel
2,7,Butaritari,KI,82.65,few clouds,3.0707,172.7902,Isles Sunset Lodge
3,12,Celestun,MX,88.56,clear sky,20.8667,-90.4000,Posada Lilia
4,14,Cayenne,GF,87.10,broken clouds,4.9333,-52.3333,Hôtel Le Dronmi
5,17,Kapaa,US,80.92,overcast clouds,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
6,25,Ammon,US,84.25,clear sky,43.4696,-111.9666,"Holiday Inn Express & Suites Idaho Falls, an I..."
7,29,Sao Filipe,CV,80.15,few clouds,14.8961,-24.4956,Hotel Xaguate
8,32,Mandera,KE,87.37,broken clouds,3.9366,41.8670,Kornesh Hotel and Lodging
9,33,Naze,JP,78.08,scattered clouds,28.3667,129.4833,オーベルジュ・アガルイ


In [3]:
# 2. Using the template add the city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>

<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 3a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in vacation_df.iterrows()]

# 3b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = vacation_df[["Lat", "Lng"]]

In [4]:
# 4a. Add a marker layer for each city to the map.
locations = vacation_df[["Lat", "Lng"]]
max_temp = vacation_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 4b. Display the figure
fig


Figure(layout=FigureLayout(height='420px'))

In [5]:
# From the map above pick 4 cities and create a vacation itinerary route to travel between the four cities. 
# 5. Create DataFrames for each city by filtering the 'vacation_df' using the loc method. 
# Hint: The starting and ending city should be the same city.

vacation_start = vacation_df.loc[vacation_df["City"] == "Hasaki"]
vacation_end = vacation_df.loc[vacation_df["City"] == "Hasaki"]
vacation_stop1 = vacation_df.loc[vacation_df["City"] == "Katsuura"]
vacation_stop2 = vacation_df.loc[vacation_df["City"] == "Shimoda"] 
vacation_stop3 = vacation_df.loc[vacation_df["City"] == "Tsuruga"] 

In [6]:
# 6. Get the latitude-longitude pairs as tuples from each city DataFrame using the to_numpy function and list indexing.
start = vacation_start["Lat"].to_numpy()[0], vacation_start["Lng"].to_numpy()[0]
end = vacation_end["Lat"].to_numpy()[0], vacation_end["Lng"].to_numpy()[0]
stop1 = vacation_stop1["Lat"].to_numpy()[0], vacation_stop1["Lng"].to_numpy()[0] 
stop2 = vacation_stop2["Lat"].to_numpy()[0], vacation_stop2["Lng"].to_numpy()[0] 
stop3 = vacation_stop3["Lat"].to_numpy()[0], vacation_stop3["Lng"].to_numpy()[0] 

In [7]:
# 7. Create a direction layer map using the start and end latitude-longitude pairs,
# and stop1, stop2, and stop3 as the waypoints. The travel_mode should be "DRIVING", "BICYCLING", or "WALKING".
fig = gmaps.figure()
city_itinerary = gmaps.directions_layer(
    start, end, waypoints=[vacation_stop1, vacation_stop2, vacation_stop3], 
    travel_mode="DRIVING")

fig.add_layer(city_itinerary)
fig

ValueError: too many values to unpack (expected 2)

In [8]:
# 8. To create a marker layer map between the four cities.
#  Combine the four city DataFrames into one DataFrame using the concat() function.
itinerary_df = pd.concat([vacation_start, vacation_stop1, vacation_stop2, vacation_stop3],ignore_index=True)
itinerary_df

,City_ID,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
0,129,Hasaki,JP,77.40,overcast clouds,35.7333,140.8333,Hotel Sunrise Choshi
1,445,Katsuura,JP,84.31,overcast clouds,35.1333,140.3000,Katsuura Hotel Mikazuki
2,518,Shimoda,JP,82.51,overcast clouds,34.6667,138.9500,Shimoda Tokyu Hotel
3,124,Tsuruga,JP,81.64,moderate rain,35.6455,136.0558,New Sunpia Tsuruga


In [10]:
# 9 Using the template add city name, the country code, the weather description and maximum temperature for the city. 
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in itinerary_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = itinerary_df[["Lat", "Lng"]]

In [12]:
# 11a. Add a marker layer for each city to the map.
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig = gmaps.figure()
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))